# Import Packages

In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.optim import AdamW, Adam
from models.mobileViTCA import make_model
from models.PANN import Wavegram_Logmel_Cnn14
from dataset import Loader
from sklearn.model_selection import train_test_split
from vanilla_kd import VanillaKD

%load_ext tensorboard

# Loading Models

## Teacher Model

In [2]:
teacher = Wavegram_Logmel_Cnn14()

# Loading weights from AudioSet pretrained model
state_dict = torch.load(r'C:\Users\jdcn\Desktop\PSLA\Wavegram_Logmel_Cnn14_mAP=0.439.pth')
teacher.load_state_dict(state_dict['model'])

c:\Users\jdcn\anaconda3\envs\tcc\lib\site-packages\torchlibrosa\stft.py:193: FutureWarning: Pass size=1024 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  fft_window = librosa.util.pad_center(fft_window, n_fft)


<All keys matched successfully>

In [3]:
# Change Classifier Layer
in_features = teacher.fc_audioset.in_features

teacher.fc_audioset = nn.Linear(in_features,4)

## Student Model

In [4]:
student = make_model(net='xxs', num_classes = 4, patch_size=(4,4), patch_stride=(3,3))

## Models Hyperparams

In [5]:
# Learning Rate
Lr = 0.0001
# Optimizer
teacher_optim = Adam(teacher.parameters(), lr=Lr, weight_decay=0)
student_optim = Adam(student.parameters(), lr=Lr, weight_decay=0)

# Dataset

In [6]:
train_df = pd.read_csv(r'D:\dataset\Train.csv')
test_df = pd.read_csv(r'D:\dataset\Test.csv')

### Creating Validation Dataframe

In [7]:
targets = train_df['id']

# Split Into 90% for training and 10% for test
train_idx, val_idx = train_test_split(np.arange(len(targets)),test_size=0.1,shuffle=True,
                                       stratify=targets,random_state=42)

val_df = train_df.iloc[val_idx].reset_index(drop=True)
train_df = train_df.iloc[train_idx].reset_index(drop=True)

### Creating Dataloader

In [8]:
train_ds = Loader(train_df, r'D:\dataset\Train', mix_up=True)
val_ds = Loader(val_df, r'D:\dataset\Train', mix_up=False)
test_ds = Loader(test_df, r'D:\dataset\Test', mix_up=False)

In [9]:
train_loader = torch.utils.data.DataLoader(train_ds, batch_size=2, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_ds, batch_size=2, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_ds, batch_size=2, shuffle=False)

# Knwoledge Distillation 

In [10]:
torch.manual_seed(1)

num_epoch=5
# Distillation instance
distiller = VanillaKD(teacher, student, train_loader, val_loader, test_loader, teacher_optim, student_optim) 

## Train Teacher

In [11]:
# Train Teacher 
distiller.train_teacher(epochs=num_epoch, plot_losses=True, save_model=True)      

Training Teacher... 


RuntimeError: CUDA out of memory. Tried to allocate 144.00 MiB (GPU 0; 4.00 GiB total capacity; 2.44 GiB already allocated; 0 bytes free; 2.49 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

## Train Student

In [ ]:
# Train Student
distiller.train_student(epochs=num_epoch, plot_losses=True, save_model=True) 

In [ ]:
%tensorboard --logdir=Experiments